In [14]:
# Read data_clean
import pandas as pd

df = pd.read_json('./data_clean/data_clean.json', orient="records")

In [19]:
def preprocess_row(row):
    # Unir las características adicionales en una sola cadena de texto
    genre = row["genres"]
    keywords = row["keywords"]

    combined_text = f"{keywords} [SEP] Genre: {genre}"
    return combined_text

In [28]:
import torch
from transformers import BertModel, BertTokenizer
import pandas as pd
import faiss
import numpy as np
from tqdm import tqdm  # Importar tqdm para la barra de progreso
import time  # Importar time para calcular el tiempo transcurrido

# Cargar modelo y tokenizer
model_name = "bert-base-uncased"
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertModel.from_pretrained(model_name)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


# Función para obtener embeddings por lote
def get_bert_embeddings(texts, batch_size=32):
    embeddings = []
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)  # Mover el modelo a GPU si está disponible

    # Inicializar tqdm para la barra de progreso
    pbar = tqdm(total=len(texts), desc="Processing texts", unit="texts")
    start_time = time.time()  # Tiempo de inicio del procesamiento

    for i in range(0, len(texts), batch_size):
        batch_texts = texts[i : i + batch_size]
        tokens = tokenizer(
            batch_texts, padding=True, truncation=True, return_tensors="pt"
        ).to(device)
        with torch.no_grad():
            outputs = model(**tokens)
        batch_embeddings = outputs.last_hidden_state.mean(dim=1)
        embeddings.append(batch_embeddings)

        # Actualizar la barra de progreso
        pbar.update(len(batch_texts))

        # Calcular tiempo transcurrido y estimar tiempo restante
        elapsed_time = time.time() - start_time
        texts_per_sec = (i + len(batch_texts)) / elapsed_time
        remaining_time = (len(texts) - (i + len(batch_texts))) / texts_per_sec

        # Actualizar descripción de la barra de progreso con el tiempo estimado
        pbar.set_postfix({"ETA": f"{remaining_time:.1f} sec"})

    pbar.close()  # Cerrar la barra de progreso al finalizar
    return torch.cat(embeddings, dim=0).cpu().numpy()

In [26]:
df['combined'] = df.apply(preprocess_row, axis=1)
combined_texts = df['combined'].tolist()

In [30]:
# Obtener embeddings
embeddings_np = get_bert_embeddings(combined_texts, batch_size=32)

8637     The St. Valentine's Day Massacre
423                          A Bronx Tale
2801                            The Limey
38914                        Café Society
289                Leon: The Professional
13825                           Dillinger
11487                   The Good Shepherd
3221                            Key Largo
8352                             Scarface
Name: title, dtype: object


In [49]:
# Crear el índice en FAISS
d = embeddings_np.shape[1]
index = faiss.IndexFlatL2(d)
index.add(embeddings_np)

# Función para obtener recomendaciones con FAISS
import numpy as np

def get_recommendations_faiss(titles, df, index, embeddings, k=10):
    # Obtener los índices de las películas que coinciden con los títulos
    indices = [df[df['title'] == title].index[0] for title in titles]
    
    # Obtener los embeddings promedio de las películas
    avg_embedding = np.mean(embeddings[indices], axis=0, keepdims=True)
    
    # Buscar los k vecinos más cercanos al embedding promedio
    D, I = index.search(avg_embedding, k)
    
    # I[0][1:] contiene los índices de las películas más similares (excluyendo las ingresadas)
    movie_indices = I[0][1:]
    
    # Devolver los títulos de las películas recomendadas
    return df['title'].iloc[movie_indices].tolist()


# Ejemplo de uso
recommended_movies_faiss = get_recommendations_faiss(['Ratatouille',"Memento"], df, index, embeddings_np, k=15)
print(recommended_movies_faiss)


['Memento', 'Ratatouille', 'The Moustache', 'New York Stories', 'Wild at Heart', 'The Skeleton Key', 'Interstate 60', 'In Her Shoes', 'New Best Friend', 'Just Like a Woman', 'The Vagrant', 'Secret Window', 'Frailty', 'Home for the Holidays']


In [51]:
# Guardar embeddings y el índice en FAISS
np.save('./assets/embeddings.npy', embeddings_np)

d = embeddings_np.shape[1]
index = faiss.IndexFlatL2(d)
index.add(embeddings_np)
faiss.write_index(index, './assets/embedding_index.faiss')